# PyKX Query Components
Query all TorQ Components.


In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
import boto3
import json
import datetime

import pykx as kx

from managed_kx import *
from env import *

from clusters import *

GATEWAY='gateway1'

In [ ]:
# Using credentials and create service client
session = boto3.Session()

# create finspace client
client = session.client(service_name='finspace')

# List of Clusters

In [ ]:
cdf = get_clusters(client, environmentId=ENV_ID)

all_clusters = [d['name'] for d in clusters if 'name' in d]

if cdf is not None:
    cdf = cdf[cdf['clusterName'].isin(all_clusters)]
#    cdf = cdf[cdf['status'].isin(['RUNNING'])]

display(cdf)

# Query the system through the Gateway

In [ ]:
conn = get_pykx_connection(client, 
    environmentId=ENV_ID, clusterName=GATEWAY, 
    userName=KDB_USERNAME, boto_session=session)

queries = [
    '.gw.syncexec["meta trades";`rdb]',
    '.gw.syncexec["meta trades";`hdb]',
    '.gw.syncexec["select sum size by sym from trades";`rdb]',
    '.gw.syncexec["count trades";`rdb]',
    '.gw.syncexec["select [5] from trades";`rdb]',
    '.gw.syncexec["select [-5] from trades";`rdb]',
    '.gw.syncexec["select count i by date from trades";`hdb]', # FIXED
    '.gw.syncexec["select sum size, max price by 0D00:05 xbar time from trades where sym=`IBM";`hdb`rdb]',
    '.gw.syncexecj["select sum size by sym from trades";`rdb`hdb;sum]',
]

for q in queries:
    print(100*"=")
    print(f"Query: {q}")
    print(100*"-")
    display( conn(q).pd() )

In [ ]:
print( f"Last Run: {datetime.datetime.now()}" )